## Permutation

Permutation group element for integer $n$

In [1]:
from __future__ import annotations
from itertools import permutations

In [3]:
per_test= Permutation(5, [2,5,4,3,1])

In [4]:
per_inverse = per_test.inverse()
print(per_test.plist)
print(per_inverse.plist)

[2, 5, 4, 3, 1]
[5, 1, 4, 3, 2]


In [5]:
(per_inverse * per_test).plist

[1, 2, 3, 4, 5]

## permutation notation
Cauchy: 2 line notation

Cyclic, Canonical Cyclic

In [6]:
print(per_test.notation_cauchy())

[1, 2, 3, 4, 5]
[2, 5, 4, 3, 1]


In [7]:
print(per_test.notation_cyclic())
print(per_inverse.notation_cyclic())

[[1, 2, 5], [3, 4]]
[[1, 5, 2], [3, 4]]


In [9]:
print(per_test.notation_cyclic(canonical=True, string= True))
print(per_inverse.notation_cyclic(canonical=True, string=True))

(43)(512)
(43)(521)


In [2]:
class Permutation:
    @classmethod
    def get_permutations(cls, n, per=False): # get list of permutations for given 'n'
        per = permutations(range(1, n+1), n)
        if per:
            return [cls(n, p) for p in per]
        else:
            return [ p for p in per]
    @classmethod
    def reverse_permutation(cls, n):
        return cls(n, range(n, 0, -1)) 


    def __init__(self, n, plist):
        if len(plist) != n:
            raise ValueError(f"{n} must be same with len(plist) = {len(plist)}")

        if sum(plist) != int(n* (n+1)/2):
            raise ValueError(f"plist does not satisfy permutation proeprty.\n All [0, n-1] values must be in plist. \n {plist}")

        self.n = n
        self.plist = plist
    def __getitem__(self, key):
        if type(key) != int:
            raise ValueError(f"key must be an integer type element: {key}")
        if key < 1 or key > self.n:
            raise IndexError(f"{key} must be in [1, {self.n}] range.")
        
        return self.plist[key-1]
    def __mul__(self, other: Permutation) -> Permutation:
        if self.n != other.n:
            raise ValueError(f"{self.n} and {other.n} are not same.")
        
        rlist = [other[x] for x in self.plist]
        return Permutation(self.n, rlist)

    def index_mul(self, other, oper=False):

        rlist = [self[x] for x in other.plist]
        if oper:
            self.plist= rlist
        else:
            return Permutation(self.n, rlist)

    def index_mul_partial(self, sub_permutation, oper = False): #Work on indexing
        if not isinstance( sub_permutation, Permutation):
            raise ValueError(f"Given parameter must be \'Permutation\' object. \n Current object:{type(sub_permutation)}")
        if self.n %sub_permutation.n != 0:
            raise ValueError(f"Sub permutation must have a divisor of main permuatain size as its size\n main:{self.n}, sub:{sub_permutation.n}")

        n = int(self.n / sub_permutation.n)
        m = sub_permutation.n
        
        rlist = []
        for i in range(0,n):
            tem_rlist = [self.plist[x+m*i -1] for x in sub_permutation.plist]
            rlist = rlist + tem_rlist
        
        if oper:
            self.plist = rlist
        else:
            return Permutation(self.n, rlist)

    def permute_to_list_index(self, li):
        if len(li) != self.n:
            raise ValueError(f"{len(li)} ! = {self.n}")
        
        rlist = [li[x-1] for x in self.plist ]
        return rlist

    def inverse(self)-> Permutation:
        ilist= [self.plist.index(x)+1 for x in range(1, self.n+1)]
        return Permutation(self.n, ilist)
    
    def notation_cauchy(self):
        return f"{list(range(1, self.n+1))}\n{self.plist}"

    
    def __canocial_order(self, cyclist):
        c_clist_1 = []
        c_clist_2 = []
        for li in cyclist:
            n = len(li)
            nmax = li.index(max(li))
            c_clist_1.append([li[x] for x in range(nmax-n, nmax)])
        maxindex = [x[0] for x in c_clist_1]
        s_index = sorted(maxindex)

        for i in s_index:
            c_clist_2.append(c_clist_1[maxindex.index(i)])
        
        return c_clist_2

    def notation_cyclic(self, canonical=False, string= False, sep=''):
        cycliclist = []
        index = list(range(1, self.n+1))
        i =1

        while True:
            ilist = []
            ilist.append(i)
            index.remove(i)
            
            sig_i = self[i]
            while sig_i in index:
                ilist.append(sig_i)
                index.remove(sig_i)
                sig_i = self[sig_i]
            
            cycliclist.append(ilist)

            if len(index) > 0:
                i = min(index)
            else: 
                break
        if canonical:
            cycliclist = self.__canocial_order(cycliclist)
        
        if string:
            cycstr = ''
            for li in cycliclist:
                cycstr = cycstr+ '('+f'{sep}'.join(map(str,li))+')'
            return cycstr
        else:
            return cycliclist



In [18]:
testlist = [i for i in range(1,8+1)]

In [19]:
testlist

[1, 2, 3, 4, 5, 6, 7, 8]

In [11]:
def split_list(li, n):
    if len(li) %n !=0:
        raise ValueError(f"The length of sublist, {n}, must be a divider of original list, {len(li)}. ")
    
    rlist =[]
    for i in range(0, int(len(li)/n)):
        ni = n*i
        rlist.append([li[ni: ni+n]][0])

    return rlist
def sig_rearrange(nn, ns, split=False): #1
        if ns == 1:
            return range(0,nn)
        n_l = nn*ns

        nlist = [i+1 for i in range(0, n_l)]
        nlist_splited = split_list(nlist, int(ns/2)) if ns != 1 else nlist
        rlist=[]

        if split:
            for i in range(0, nn):
                print(nn)
                print(2*nn-i-1)
                rlist.append(nlist_splited[i]+nlist_splited[2*nn-i-1])
        else:
            for i in range(0,nn):
                rlist = rlist + nlist_splited[i] + nlist_splited[2*nn-i-1]

        return rlist

In [12]:
sig_rearrange(1,1, split=True)

1
1


IndexError: list index out of range

In [18]:
range(0,3)[1::2]

range(1, 3, 2)

In [54]:
nlist_splited

[[1, 2], [3, 4], [5, 6], [7, 8]]

In [20]:
fold_arrange ={ # From left-top to right-bottom
    4: [
            [4,1], # Front page
            [2,3]  # Back page
        ],
    8: [
        [8,1,5,4],
        [2,7,3,6]
    ],
    12: [
        [12,1,9,4,8,5],
        [2,11,3,10,6,7]
    ],
    16: [
        [16,1,4,13,9,8,5,12],
        [10,7,6,11,15,2,3,14]
    ],
    24: [
        [24, 1, 12, 13, 21, 4, 9, 16, 20, 5, 8, 17],
        [14, 11, 2, 23, 15, 10, 3, 22, 18, 7, 6, 19]
    ],
    32: [
        [44,21,28,37,40,25,24,41,53,12,5,60,57,8,9,56,52,13,4,61,64,1,16,49,45,20,29,36,33,32,17,48],
        [46,19,30,35,34,31,18,47,51,14,3,62,63,2,15,50,54,11,6,59,58,7,10,55,43,22,27,38,39,26,23,42]
    ]
}
def sig_layout(n:int)->tuple:
    if type(n) != int:
        raise ValueError("n is not an integer")
    if n==1:
        return (1,1) 
    elif n<4 or n%4 !=0:
        raise ValueError(f"n:{n} must be a positive integer that multiple of 4.")
    if n%3 ==0:
        i = log2(n) - log2(3) -1
        return(3, int(2**i))
    else:
        i = int(log2(n/4))
        if i%2 :
            k = kp = int((i+1)/2)
        else:
            k = int(i/2)
            kp = k+1
        return (int(2**k), int(2**kp)) 

In [62]:
sig_layout(256)

(8, 16)

In [21]:
import numpy as np
import sys
from typing import Union, Tuple, NoReturn
sys.path.append("..")
sys.path.append(".")
from math import log, log2
from permutation import Permutation

In [19]:
def __fold_matrix_update(n:int, matrix:np.ndarray) -> np.ndarray:
    n_1 = np.flip(matrix.T, axis=0)
    len_n = len(n_1[0])
    l = int(len_n/2)
    rows =[]
    for row in n_1:
        r_split = np.split(row,l)
        row_appended = []
        for tu in r_split:
            tem = np.array([n-tu[0] +1,n-tu[1] +1])
            row_appended.append(np.insert(tem, 1, tu))    
        rows.append(np.concatenate(row_appended, axis=None))     
    return np.stack(rows)

def fold_arrange_n(n, per=False)->Union[list, Permutation]:
    if n==2:
        if per:
            return Permutation(2, [1,2])
        else:
            return [[1],[2]]
    if n % 4 !=0:
        raise ValueError("Fold sheets must be 4*2^k for k= 0, 1, 2, .... \n Current value is {n}")
    
    if n < 64:
        fn = fold_arrange[n]
        if per:
            return Permutation(n, fn[0]+fn[1])
        else:
            return fn
    else:
        n_iter = int(log(n/16,2))
        n_i = 32
        per_n_1 = [fold_arrange[n_i][0], fold_arrange[n_i][1]]
        #permutation to matrix
        layout_n_1 = sig_layout(n_i)
        front_matrix = np.array(per_n_1[0]).reshape(layout_n_1)
        back_matrix = np.array(per_n_1[1]).reshape(layout_n_1)
        for i in range(0, n_iter):
            n_i = 2*n_i
            front_matrix = __fold_matrix_update(n_i, front_matrix)
            back_matrix = __fold_matrix_update(n_i, back_matrix)   
    per_fn = np.concatenate(front_matrix).tolist() 
    per_bn = np.concatenate(back_matrix).tolist()
    
    if per:
        return Permutation(n, per_fn+per_bn )
    else:
        return [per_fn, per_bn]

In [77]:
def transpose(matrix:list)->list:
    size_rows = len(matrix)
    size_columns = len(matrix[0])

    t = list()
    for i in range(0, size_columns):
        t.append([])
    
    for i in range(0, size_rows):
        for j in range(0,size_columns):
            t[j].append(matrix[i][j])
    
    return t
def flip(matrix): #axis=0
    size = len(matrix)

    f = list()
    for i in range(0, size):
        f.append(matrix[size-i-1])
    
    return f 
def split_list(li: list, n:int, mode='l')->list:
    """
    :param li: list, list to be splited.
    :param n: int, The length of sublist. It must be a divider of the length of :param:`li` list.
    :param mode: str, The mode of split, `l`: length of sublist, `n`: number of sublist
    """
    if mode != 'l' and mode != 'n':
        raise ValueError('The \'mode\' parameter must be \'l\' or \'n\', current = {mode}')

    num = n
    l_li = len(li)
    if mode =='n':
        if l_li%num !=0:
            raise ValueError('The length of the given list and the sublist length must have a divider relationship.')
        num = int(l_li/num)
        mode ='l'

    if mode == 'l':
        if num <=1:
            return li
        if len(li) %num !=0:
            raise ValueError(f"The length of sublist, {num}, must be a divider of original list, {len(li)}. ")

        rlist =[]
        for i in range(0, int(l_li/num)):
            ni = num*i
            rlist.append([li[ni: ni+num]][0])

    return rlist 
def concatenate(lists):
    length = len(lists)

    rlist = list()
    for i in range(0, length):
        for element in lists[i]:
            rlist.append(element)
    
    return rlist

def reshape(list_1d, shape):
    size_1d = len(list_1d)
    if size_1d != shape[0] * shape[1]:
        raise ValueError("The list length and shape are not matched each other.")
    return split_list(list_1d, shape[0], mode='n')

In [79]:
test_transpose = [[1,2],[3,4],[5,6],[7,8]]
test_transpose2 = [ [1,2,3,4,5,6],
                   [7,8,9,10,11,12],
                   [13,14,15,16,17,18],
                   [19,20,21,22,23,24],
                   [25,26,27,28,29,30],
                   [31,32,33,34,35,36]]

In [74]:
test_transpose

[[1, 2], [3, 4], [5, 6], [7, 8]]

In [80]:
transpose(test_transpose)

[[1, 3, 5, 7], [2, 4, 6, 8]]

In [81]:
transpose(test_transpose2)

[[1, 7, 13, 19, 25, 31],
 [2, 8, 14, 20, 26, 32],
 [3, 9, 15, 21, 27, 33],
 [4, 10, 16, 22, 28, 34],
 [5, 11, 17, 23, 29, 35],
 [6, 12, 18, 24, 30, 36]]

In [55]:
split_list([1,2,3,4,5,6], 3, mode='n')

[[1, 2], [3, 4], [5, 6]]

In [37]:
def __fold_matrix_update_pre(n:int, matrix:np.ndarray) -> np.ndarray:
    n_1 = np.flip(matrix.T, axis=0)
    len_n = len(n_1[0])
    l = int(len_n/2)
    rows =[]
    for row in n_1:
        r_split = np.split(row,l)
        row_appended = []
        for tu in r_split:
            tem = np.array([n-tu[0] +1,n-tu[1] +1])
            row_appended.append(np.insert(tem, 1, tu))    
        rows.append(np.concatenate(row_appended, axis=None))     
    return np.stack(rows)

In [ ]:
def fold_arrange_n(n, per=False)->Union[list, Permutation]:
    if n==2:
        if per:
            return Permutation(2, [1,2])
        else:
            return [[1],[2]]
    if n % 4 !=0:
        raise ValueError("Fold sheets must be 4*2^k for k= 0, 1, 2, .... \n Current value is {n}")
    
    if n < 64:
        fn = fold_arrange[n]
        if per:
            return Permutation(n, fn[0]+fn[1])
        else:
            return fn
    else:
        n_iter = int(log(n/16,2))
        n_i = 32
        per_n_1 = [fold_arrange[n_i][0], fold_arrange[n_i][1]]
        #permutation to matrix
        layout_n_1 = sig_layout(n_i)
        front_matrix = reshape(per_n_1[0], layout_n_1)
        back_matrix = reshape(per_n_1[1], layout_n_1)
        for i in range(0, n_iter):
            n_i = 2*n_i
            front_matrix = __fold_matrix_update(n_i, front_matrix)
            back_matrix = __fold_matrix_update(n_i, back_matrix)   
    per_fn = concatenate(front_matrix)
    per_bn = concatenate(back_matrix)
    
    if per:
        return Permutation(n, per_fn+per_bn )
    else:
        return [per_fn, per_bn]

In [64]:
a= [1,4]
a.insert(1,[2,3])

In [69]:
concatenate([[1,2,3,4],[5,6,7,8,9],[10,11],[12,13,14,15,16,17,18]])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [70]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

In [71]:
np.stack([a,b])

array([[1, 2, 3],
       [4, 5, 6]])

In [ ]:
def __fold_matrix_update(n:int, matrix:np.ndarray) -> np.ndarray:
    n1 = flip(transpose(matrix))
    len_n = len(n1)
    l = int(len_n/2)
    rows =[]
    for row in n1:
        r_split = split_list(row, l, mode='n')
        row_appended = []
        for tu in r_split:
            tem = [n-tu[0] +1,n-tu[1] +1]
            row_appended.append(tem.insert(1, tu[1]))
            row_appended.append(tem.insert(1, tu[0]))
        rows.append(concatenate(row_appended))
    return rows